# **1. 라이브러리 임포트**

In [34]:
import pandas as pd
import numpy as np

# 위도와 경도 간의 거리를 계산
from haversine import haversine, haversine_vector
# 지리적 위치 정보를 얻기 위한 
from geopy.geocoders import Nominatim 
# 문자열을 파이썬 데이터 구조로 변환하기 위한
import ast 

# **2. 데이터 로드**

## 1) 아파트 데이터
- 지하철역, 병원 수, 대학까지 추가된 데이터

In [45]:
apartment = pd.read_csv("apartment4.csv")

# apartment의 'location'열을 문자열에서 튜플로 변환 후 위치 데이터 추출
apartment.loc[:, 'location'] = apartment['location'].apply(ast.literal_eval)
apartment.head(1)

,Unnamed: 0,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,...,입주예정일,공급액(만원),대형건설사,location,지하철역,지하철역_거리(km),1차병원,2차병원,3차병원,대학교_수
0,0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,...,202501,"132,040",YES,"(37.5228859, 127.140539)",9호선 둔촌오륜역,0.6223,46,16,1,1


## 2) 공원 데이터

In [46]:
park = pd.read_csv('전국도시공원정보표준데이터.csv', encoding='euc-kr')
park.head(1)
park.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18064 entries, 0 to 18063
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          18064 non-null  object 
 1   공원명           18064 non-null  object 
 2   공원구분          18064 non-null  object 
 3   소재지도로명주소      3651 non-null   object 
 4   소재지지번주소       17901 non-null  object 
 5   위도            18052 non-null  float64
 6   경도            18053 non-null  float64
 7   공원면적          18064 non-null  float64
 8   공원보유시설(운동시설)  5014 non-null   object 
 9   공원보유시설(유희시설)  6439 non-null   object 
 10  공원보유시설(편익시설)  4992 non-null   object 
 11  공원보유시설(교양시설)  744 non-null    object 
 12  공원보유시설(기타시설)  2706 non-null   object 
 13  지정고시일         15088 non-null  object 
 14  관리기관명         16703 non-null  object 
 15  전화번호          17009 non-null  object 
 16  데이터기준일자       18064 non-null  object 
 17  제공기관코드        18064 non-null  object 
 18  제공기관명         18064 non-nu

# **3. 공원 데이터 전처리**

In [47]:
# <공원 '공원구분', '제공기관명' 데이터 확인>

# print('공원구분 유니크 값 : ', park['공원구분'].unique()) 
# print('제공기관명 유니크 값 : ', park['제공기관명'].unique()) 

## 1) '공원구분' 수정
- 삭제할 값 -> '묘지공원', 도시농업공원, '가로공원' '공공공지'
- 수정할 값 -> 도시자연공원구역(도시자연공원), 어린인공원(어린이공원)

In [48]:
# (수정 전) 공원구분
park['공원구분'].value_counts()

어린이공원       9366
근린공원        4023
소공원         2781
기타           568
문화공원         404
수변공원         337
체육공원         258
역사공원         193
마을마당          49
묘지공원          39
기타공원          30
도시농업공원         7
도시자연공원구역       2
주제공원           2
도시자연공원         2
어린인공원          1
가로공원           1
공공공지           1
Name: 공원구분, dtype: int64

In [49]:
# 삭제
park_df = park[~park['공원구분'].isin(['묘지공원', '도시농업공원', '가로공원', '공공공지', '기타', '기타공원'])]

# 수정
park_df['공원구분'] = park_df['공원구분'].replace({'어린인공원': '어린이공원', 
                                            '도시자연공원구역': '도시자연공원'})

# (수정 후) 공원 구분 
park_df['공원구분'].value_counts()

/var/folders/ht/vmxckhk151s1x5lpr_f5drm40000gn/T/ipykernel_1819/3181141078.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park_df['공원구분'] = park_df['공원구분'].replace({'어린인공원': '어린이공원',


어린이공원     9367
근린공원      4023
소공원       2781
문화공원       404
수변공원       337
체육공원       258
역사공원       193
마을마당        49
도시자연공원       4
주제공원         2
Name: 공원구분, dtype: int64

## 2) '제공기관명' 필터링
- 주소는 null값이 존재 -> 제공기관명으로 '서울', '경기', '인천' 지역 필터링
- '서울', '경기', '인천' 지역의 공원 데이터만 선택

In [50]:
# (수정 전) 서울 | 경기 | 인천을 제외한 타지역 존재 -> 필터링 필요함
print('제공기관명 필터링 전 shape :', park_df.shape, '\n')
park_df['제공기관명'].value_counts()

제공기관명 필터링 전 shape : (17418, 19) 



충청남도        944
경상남도 창원시    466
경기도 평택시     458
광주광역시       425
충청북도 청주시    424
           ... 
경상북도 청송군      1
부산관광공사        1
전라남도 구례군      1
전라북도 순창군      1
경상남도 합천군      1
Name: 제공기관명, Length: 243, dtype: int64

In [51]:
# '제공기관명' 값이 '서울', '경기', '인천'을 포함하는 행 필터링
park_df = park_df[park_df['제공기관명'].str.contains('서울|경기|인천')]

print('제공기관명 필터링 후 shape :', park_df.shape, '\n')
print('제공기관명 필터링 후 :', park_df['제공기관명'].value_counts())

제공기관명 필터링 후 shape : (6645, 19) 

제공기관명 필터링 후 : 경기도 평택시      458
경기도 수원시      335
경기도 화성시      300
경기도 용인시      274
경기도 고양시      253
            ... 
인천시설공단        18
경기도 연천군       17
경기도 가평군       15
인천광역시 동구      14
인천광역시 강화군      4
Name: 제공기관명, Length: 67, dtype: int64


## 3) NaN 값 제거
- '위도'와 '경도' 컬럼에 결측값이 있는 행 삭제 후 확인

In [52]:
# '위도'와 '경도' 컬럼에 NaN 값이 있는 행 삭제
park_df = park_df.dropna(subset=['위도', '경도'])
print('결측값 제거 후 shape :', park_df.shape, '\n')

# park_df[park_df['위도'].isna() | park_df['경도'].isna()]

결측값 제거 후 shape : (6627, 19) 



## 4) 불필요한 컬럼 삭제
- 분석에 필요 없는 컬럼 삭제

In [53]:
park_df.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [54]:
# 특정 컬럼 삭제
park_df.drop(columns=['관리번호', '소재지도로명주소', '소재지지번주소', '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)', 
                      '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드'], inplace=True)

print('컬럼 삭제 후 공원 전체 shape : ', park_df.shape, '\n')

# park_df.head(1)

컬럼 삭제 후 공원 전체 shape :  (6627, 6) 



# 5) '위도'와 '경도'를 튜플로 묶어 새로운 'location' 컬럼 생성

In [56]:
park_df['location'] = tuple(zip(park_df['위도'], park_df['경도']))
park_df.head(1)
# park_df.info()

,공원명,공원구분,위도,경도,공원면적,제공기관명,location
0,승지공원,근린공원,37.371378,126.813132,10842.0,경기도 시흥시,"(37.371378, 126.813132)"


## 6) 아파트와 공원의 위치 데이터 추출

In [58]:
park_location = list(park_df['location'])
apartment_location = list(apartment['location'])

# park_location
# apartment_location

# **4. 아파트와 가장 가까운 공원 개수 계산**
- haversine_vector 함수를 사용하여 공원과 아파트 사이의 거리 계산
- 아파트에서 1km 이내에 있는 공원 개수 계산

In [60]:
distance_to_park = haversine_vector(park_location, apartment_location, comb=True)

# 아파트에서 1km 이내에 있는 공원 개수 계산
num_close_park = [np.sum(distance <= 1) for distance in distance_to_park]

# 결과를 아파트 데이터프레임에 추가    
apartment['공원'] = num_close_park

In [62]:
# 공원 변수 생성 확인
apartment.head(3)

,Unnamed: 0,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,...,공급액(만원),대형건설사,location,지하철역,지하철역_거리(km),1차병원,2차병원,3차병원,대학교_수,공원
0,0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,...,"132,040",YES,"(37.5228859, 127.140539)",9호선 둔촌오륜역,0.6223,46,16,1,1,3
1,1,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,...,"131,280",YES,"(37.5228859, 127.140539)",9호선 둔촌오륜역,0.6223,46,16,1,1,3
2,2,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,...,"131,240",YES,"(37.5228859, 127.140539)",9호선 둔촌오륜역,0.6223,46,16,1,1,3


# **5. 생성된 df csv로 저장**

In [63]:
apartment.to_csv('apartment5.csv')